<a href="https://colab.research.google.com/github/a2subedi/streamlit-dssg/blob/main/Copy%20of%20beginner-helper-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Imports

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product

import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np

import math
import random

import warnings
# warnings.filterwarnings('ignore')

In [3]:
initial_df = pd.read_csv("/content/drive/MyDrive/dataset.csv")

In [4]:
# get required indicators
INDICATORS_LIST = [
  "Government expenditure on education, total (% of GDP)",
  "Labor force, total",
  "Literacy rate, adult total (% of people ages 15 and above)",
  "Population ages 15-64 (% of total population)",
  "School enrollment, primary (% gross)",
  "School enrollment, secondary (% gross)",
  "School enrollment, tertiary (% gross)",
  "Unemployment, total (% of total labor force) (modeled ILO estimate)",
  "Central government debt, total (% of GDP)",
  "GDP (current US$)",
  "GDP growth (annual %)",
  "GDP per capita growth (annual %)",
  "GDP per capita, PPP (current international $)",
  "Population, total"
]

In [12]:
# Helper Functions

def get_df_from_indicators_and_country(df, indicator1, indicator2, country):
    # extract column ["Country Code", "Country Name", "Indicator Name", "year", "value"]
    df = df[["Country Code", "Country Name", "Indicator Name", "year", "value"]]
    return df

def process_one_indicator(df, indicator):
    # extract rows with indicator
    df = df[(df["Indicator Name"] == indicator) | (df["Indicator Code"] == indicator)]
    # only have year, value
    df = df[["year", "value"]]
    return df

def get_first_df_with_indicators_years_and_values(df, country):
    # extract columns ["Country Code", "Country Name", "Indicator Name", "Indicator Code", "year", "value"]
    df = df[(df["Country Name"] == country) | (df["Country Code"] == country)]
    df = df[["Country Code", "Country Name", "Indicator Name", "Indicator Code", "year", "value"]].dropna()
    # extract rows with country
    return df

def get_indicators(df):
    return df[df['Indicator Name'].isin(INDICATORS_LIST)].drop_duplicates().set_index('Indicator Code')['Indicator Name'].to_dict()

def get_countries(df):
    return df[['Country Name','Country Code']].drop_duplicates().set_index('Country Name')['Country Code'].to_dict()

def get_country_indicators(df, country):
    # Filter for the specified country
    country_df = df[(df["Country Name"] == country) | (df["Country Code"] == country)]

    # Pivot the dataframe
    pivot_df = country_df.pivot(index='year', columns='Indicator Code', values='value')

    # Reset index to make 'year' a column
    pivot_df.reset_index(inplace=True)

    return pivot_df


def drop_all_indicators_which_dont_have_enough_data(df):
    # drop counts which don't have 53 years
    description = df.describe(include='all')

    # Get the count of non-null values for each column
    non_null_counts = description.loc['count']

    # Filter columns that have exactly 53 non-null values
    columns_to_keep = ["year"] + non_null_counts[non_null_counts == 53].index.to_list()

    print (columns_to_keep)

    # Drop columns that do not have exactly 53 non-null values
    df_filtered = df[columns_to_keep].dropna()
    return df_filtered
    # print(df_filtered)

def check_df(dataset, head = 5):
    print('#'*30 + ' Shape of Dataset ' + '#'*30, end = '\n'*2)
    print(dataset.shape, end = '\n'*2)
    print('#'*30 + ' General informations about to Dataset ' + '#'*30, end = '\n'*2)
    print(dataset.info(), end = '\n'*2)
    print('#'*30 + ' First 5 Lines Of Dataset ' + '#'*30, end = '\n'*2)
    print(dataset.head(head), end = '\n'*2)
    print('#'*30 + ' NaN values of Features ' + '#'*30, end = '\n'*2)
    print(dataset.isnull().sum(), end = '\n'*2)
    print('#'*30 + ' Descriptive Statistics of Numerical Features ' + '#'*30, end = '\n'*2)
    print(dataset.describe().T, end = '\n'*2)
    print('#'*30 + ' Quantiles of Numerical Features ' + '#'*30, end ='\n'*2)
    print(dataset.describe([0,0.10, 0.25, 0.50,0.75,0.99]).T, end = '\n'*2)

In [7]:
country_code = "SAS"
indicators_dict = get_indicators(initial_df)

In [13]:
first_df = get_country_indicators(initial_df, country_code)
check_df(first_df)
second_df = drop_all_indicators_which_dont_have_enough_data(first_df)


############################## Shape of Dataset ##############################

(54, 50)

############################## General informations about to Dataset ##############################

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  54 non-null     int64  
 1   FP.CPI.TOTL.ZG        53 non-null     float64
 2   GC.DOD.TOTL.GD.ZS     29 non-null     float64
 3   NY.GDP.DEFL.KD.ZG     53 non-null     float64
 4   NY.GDP.MKTP.CD        53 non-null     float64
 5   NY.GDP.MKTP.KD.ZG     53 non-null     float64
 6   NY.GDP.PCAP.KD.ZG     53 non-null     float64
 7   NY.GDP.PCAP.PP.CD     33 non-null     float64
 8   SE.ADT.1524.LT.FE.ZS  48 non-null     float64
 9   SE.ADT.1524.LT.MA.ZS  48 non-null     float64
 10  SE.ADT.1524.LT.ZS     48 non-null     float64
 11  SE.ADT.LITR.FE.ZS     48 non-null     